In [1363]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import folium
import csv

In [1364]:
clients_df = pd.read_csv("Proyecto_Caso_Base/clients.csv")
depots_df = pd.read_csv("Proyecto_Caso_Base/depots.csv")
vehicles_df = pd.read_csv("Proyecto_Caso_Base/vehicles.csv")



In [1365]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())


   ClientID  LocationID  Demand  Longitude  Latitude
0         1           2      13 -74.098938  4.597954
1         2           3      15 -74.075571  4.687821
2         3           4      12 -74.107085  4.709494
3         4           5      15 -74.097280  4.605029
4         5           6      20 -74.164641  4.648464
   LocationID  DepotID  Longitude  Latitude  Capacity
0           1        1 -74.153536  4.743359        90
  VehicleType  Capacity  Range
0     Gas Car       130    170
1     Gas Car       140    200
2     Gas Car       120    180
3     Gas Car       100     90
4     Gas Car        70    100


In [1366]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))


In [1367]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)


Conjunto C: [1]
Capacidades CAP_i: {1: 90}


In [1368]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Demand"]))  # Esta es la demanda que solicita ese cliente

In [1369]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Capacidades Demanda: {1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15, 11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12, 19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11}


Conjunto para la distancia

In [1370]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c



In [1371]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)


In [1372]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 17.26
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 10.62
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 6.38
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 16.6
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 10.62


Procesamiento de vehiculos

In [1373]:
vehicles_df

,VehicleType,Capacity,Range
0,Gas Car,130,170
1,Gas Car,140,200
2,Gas Car,120,180
3,Gas Car,100,90
4,Gas Car,70,100
5,Gas Car,55,170
6,Gas Car,110,150
7,Gas Car,114,140


In [1374]:
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))



In [1375]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)


Vehículos: [0, 1, 2, 3, 4, 5, 6, 7]
Capacidades: {0: 130, 1: 140, 2: 120, 3: 100, 4: 70, 5: 55, 6: 110, 7: 114}
Rangos: {0: 170, 1: 200, 2: 180, 3: 90, 4: 100, 5: 170, 6: 150, 7: 140}


In [1376]:
for v in V:
    print(f"vehiculo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

vehiculo 0: Tipo=Gas Car, Capacidad=130, Rango=170
vehiculo 1: Tipo=Gas Car, Capacidad=140, Rango=200
vehiculo 2: Tipo=Gas Car, Capacidad=120, Rango=180
vehiculo 3: Tipo=Gas Car, Capacidad=100, Rango=90
vehiculo 4: Tipo=Gas Car, Capacidad=70, Rango=100
vehiculo 5: Tipo=Gas Car, Capacidad=55, Rango=170
vehiculo 6: Tipo=Gas Car, Capacidad=110, Rango=150
vehiculo 7: Tipo=Gas Car, Capacidad=114, Rango=140


# Creacion del modelo

In [1377]:
model=ConcreteModel()

#Definicion de conjuntos
model.C= Set(initialize=C) # Centros de distribucion
model.K = Set(initialize=K) # Clientes
model.V = Set(initialize=V) # Vehículos
model.N = model.C | model.K    # Todos los nodos CDs + Clientes el | hace la unicoon de los conjuntos


In [1378]:
# Parámetros
model.CAP = Param(model.C, initialize=CAP)
model.DEM = Param(model.K, initialize=DEM)
model.Q = Param(model.V, initialize=Q)
model.R = Param(model.V, initialize=R)

# Distancias entre centros y clientes
model.DIST = Param(model.C, model.K, initialize=DIST, within=NonNegativeReals)

Pf = 123.12  # COP/km (combustible)
Ft = 823     # COP/km (flete)
Cm = 700     # COP/km (mantenimiento)


### Variables de decision

In [1379]:
model.x =Var(model.C,model.K,model.V,domain=Binary) #X{i,j,l}
model.y= Var(model.C,model.V,domain=Binary) #y{i,l}
model.I=Var(model.C,domain=NonNegativeReals) # Inventario asignado a los centros de distribucion
model.u= Var(model.K,model.V,domain=NonNegativeReals) # Carga entregada en cada punto de distribucion



### Funcion objetivo

In [1380]:
def objetivo(model):
  return sum(
    (Pf+Ft+Cm)* model.DIST[i,j]*model.x[i,j,l]
    for i in model.C for j in model.K for l in model.V
  )

model.OBJ=Objective(rule=objetivo,sense=minimize)


####  Restricciones

Capacidad de los Centros de Distribución

In [1381]:
def restriccionCapacidad_CentroDistribucion(model,i):
  return model.I[i]<=model.CAP[i]

model.RestriccionCapacidad_CentroDistribucion=Constraint(model.C, rule=restriccionCapacidad_CentroDistribucion)

Satisfacción de la demanda de cada cliente

In [1382]:
def restriccion_demanda(model, j):
    return sum(model.u[j, l] for l in model.V) == model.DEM[j]

model.RestriccionDemanda = Constraint(model.K, rule=restriccion_demanda)

Capacidad del vehículo

In [1383]:
def restriccion_capacidad_vehiculo(model, l):
    return sum(model.u[j, l] for j in model.K) <= sum(model.Q[l] * model.y[i, l] for i in model.C)

model.RestriccionCapacidadVehiculo = Constraint(model.V, rule=restriccion_capacidad_vehiculo)


Asignar cada vehículo a un único centro de distribución

In [1384]:
def restriccion_asignacion_vehiculos(model, l):
    return sum(model.y[i, l] for i in model.C) == 1

model.RestriccionAsignacionVehiculo = Constraint(model.V, rule=restriccion_asignacion_vehiculos)


Rango util del vehiculo

In [1385]:
def restriccion_rango(model, l):
    return sum(model.DIST[i, j] * model.x[i, j, l] for i in model.C for j in model.K) <= model.R[l]

model.RestriccionRango = Constraint(model.V, rule=restriccion_rango)


In [1386]:
def link_xy_rule(m, i, j, l):
    return m.x[i, j, l] <= m.y[i, l]
model.link_xy = Constraint(model.C, model.K, model.V, rule=link_xy_rule)

In [1387]:
def link_ux_rule(m, j, l):
    return m.u[j, l] <= m.Q[l] * sum(m.x[i, j, l] for i in m.C)
model.link_ux = Constraint(model.K, model.V, rule=link_ux_rule)


In [1388]:
def visita_rule(m, j):
    return sum(m.x[i, j, l] for i in m.C for l in m.V) == 1
model.visita = Constraint(model.K, rule=visita_rule)

In [1389]:
def origen_rule(m, l):
    return sum(m.y[i, l] for i in m.C) == 1
model.un_origen = Constraint(model.V, rule=origen_rule)

resolver el modelo

In [1390]:
solver = SolverFactory('glpk')
# Resolver
result = solver.solve(model, tee=True)

# Mostrar el estado y el valor de la función objetivo
print("Estado:", result.solver.status)
print("Óptimo:", value(model.OBJ))


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\jorgi\AppData\Local\Temp\tmpmbje7u9e.glpk.raw --wglp C:\Users\jorgi\AppData\Local\Temp\tmp7w2v09vv.glpk.glp
 --cpxlp C:\Users\jorgi\AppData\Local\Temp\tmp530bjln_.pyomo.lp
Reading problem data from 'C:\Users\jorgi\AppData\Local\Temp\tmp530bjln_.pyomo.lp'...
C:\Users\jorgi\AppData\Local\Temp\tmp530bjln_.pyomo.lp:3551: warning: lower bound of variable 'x2' redefined
C:\Users\jorgi\AppData\Local\Temp\tmp530bjln_.pyomo.lp:3551: warning: upper bound of variable 'x2' redefined
465 rows, 393 columns, 1561 non-zeros
200 integer variables, all of which are binary
3751 lines were read
Writing problem data to 'C:\Users\jorgi\AppData\Local\Temp\tmp7w2v09vv.glpk.glp'...
3272 lines were written
GLPK Integer Optimizer 5.0
465 rows, 393 columns, 1561 non-zeros
200 integer variables, all of which are binary
Preprocessing...
192 constraint coefficient(s) were reduced
256 rows, 384 columns, 1152 non-zeros
192 int

In [1391]:
for l in model.V:
    # Crear un registro de la ruta
    ruta = []
    
    # Buscamos el centro de distribución desde el que parte el vehículo
    for i in model.C:
        if model.y[i, l].value == 1:
            ruta.append(f"CD {i}")
            break  # Solo un centro de distribución

    # Luego, buscamos a qué clientes va el vehículo
    cliente_anterior = None
    for i in model.C:
        for j in model.K:
            if model.x[i, j, l].value == 1: 
                if cliente_anterior:
                    ruta.append(f"Cliente {cliente_anterior} a Cliente {j}")
                else:
                    ruta.append(f"CD {i} a Cliente {j}")
                cliente_anterior = j 
               
    print(f"Vehículo {l} está en este punto y sigue esta ruta: {' -> '.join(ruta)}")


Vehículo 0 está en este punto y sigue esta ruta: CD 1 -> CD 1 a Cliente 1 -> Cliente 1 a Cliente 7 -> Cliente 7 a Cliente 9 -> Cliente 9 a Cliente 11 -> Cliente 11 a Cliente 15 -> Cliente 15 a Cliente 16 -> Cliente 16 a Cliente 22 -> Cliente 22 a Cliente 24
Vehículo 1 está en este punto y sigue esta ruta: CD 1 -> CD 1 a Cliente 3 -> Cliente 3 a Cliente 4 -> Cliente 4 a Cliente 5 -> Cliente 5 a Cliente 12 -> Cliente 12 a Cliente 13 -> Cliente 13 a Cliente 17 -> Cliente 17 a Cliente 20 -> Cliente 20 a Cliente 23
Vehículo 2 está en este punto y sigue esta ruta: CD 1 -> CD 1 a Cliente 14
Vehículo 3 está en este punto y sigue esta ruta: CD 1
Vehículo 4 está en este punto y sigue esta ruta: CD 1
Vehículo 5 está en este punto y sigue esta ruta: CD 1
Vehículo 6 está en este punto y sigue esta ruta: CD 1
Vehículo 7 está en este punto y sigue esta ruta: CD 1 -> CD 1 a Cliente 2 -> Cliente 2 a Cliente 6 -> Cliente 6 a Cliente 8 -> Cliente 8 a Cliente 10 -> Cliente 10 a Cliente 18 -> Cliente 18 a 

In [1392]:
for i in model.C:
    for l in model.V:
        if model.y[i, l].value == 1:
            print(f"Vehículo {l} parte desde CD {i}")


Vehículo 0 parte desde CD 1
Vehículo 1 parte desde CD 1
Vehículo 2 parte desde CD 1
Vehículo 3 parte desde CD 1
Vehículo 4 parte desde CD 1
Vehículo 5 parte desde CD 1
Vehículo 6 parte desde CD 1
Vehículo 7 parte desde CD 1


In [1393]:
mapa = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

for _, depot in depots_df.iterrows():
    folium.Marker(
        location=[depot['Latitude'], depot['Longitude']], 
        popup=f"CD {depot['DepotID']}", 
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

for _, client in clients_df.iterrows():
    folium.Marker(
        location=[client['Latitude'], client['Longitude']], 
        popup=f"Cliente {client['ClientID']}",
        icon=folium.Icon(color='green')
    ).add_to(mapa)

for l in model.V:
    ruta_coordenadas = []

    for i in model.C:
        if model.y[i, l].value == 1:
            cd_lat = depots_df[depots_df['DepotID'] == i]['Latitude'].values[0]
            cd_lon = depots_df[depots_df['DepotID'] == i]['Longitude'].values[0]
            ruta_coordenadas.append([cd_lat, cd_lon])
            break

    cliente_anterior = None
    for i in model.C:
        for j in model.K:
            if model.x[i, j, l].value == 1:
                client_lat = clients_df[clients_df['ClientID'] == j]['Latitude'].values[0]
                client_lon = clients_df[clients_df['ClientID'] == j]['Longitude'].values[0]
                ruta_coordenadas.append([client_lat, client_lon])
                
                cliente_anterior = j

    folium.PolyLine(ruta_coordenadas, color='red', weight=2.5, opacity=1).add_to(mapa)

mapa

In [1394]:
# Definimos la velocidad promedio en Bogotá para un furgón
velocidad_promedio = 40  # km/h

# Calcular el tiempo total basado en la distancia recorrida
def calcular_tiempo_total(distancia_km):
    tiempo_horas = distancia_km / velocidad_promedio 
    return tiempo_horas * 60 


def guardar_verificacion(model, archivo='verificacion_caso.csv'):
    with open(archivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence', 'ClientsServed', 
                         'DemandsSatisfied', 'TotalDistance', 'TotalTime', 'FuelCost'])

        # Recorremos los vehículos
        for l in model.V:
            vehicle_id = f"{l+1:03d}"
            
            depot_id = None
            for i in model.C:
                if model.y[i, l].value == 1:
                    depot_id = i
                    break
            
            initial_load = model.I[depot_id].value if depot_id is not None else 0
            
            route_sequence = []
            cliente_anterior = None
            total_distance = 0  # Distancia total
            for i in model.C:
                for j in model.K:
                    if model.x[i, j, l].value == 1:
                        if cliente_anterior:
                            route_sequence.append(f"C{cliente_anterior} - C{j}")
                        else:
                            route_sequence.append(f"CD {i} - C{j}")
                        cliente_anterior = j
                        total_distance += model.DIST[i, j]
            
            # Clientes atendidos
            clients_served = len(route_sequence)
            
            demands_satisfied = []
            for j in model.K:
                if cliente_anterior == j:
                    demands_satisfied.append(str(model.DEM[j])) 

            total_time = calcular_tiempo_total(total_distance)
            
            fuel_cost = total_distance * (Pf + Ft + Cm)
            
            writer.writerow([
                vehicle_id,
                depot_id,
                initial_load,
                '-'.join(route_sequence),
                clients_served,
                '-'.join(demands_satisfied),
                total_distance,
                total_time,
                fuel_cost
            ])

guardar_verificacion(model, archivo='verificacion_caso1.csv')
